In [147]:
import sys
sys.path.append('../')
import numpy as np
import pandas as pd
from grid_world.grid_world import GridWorld
from DMEIRL.DeepMEIRL_FC import DMEIRL
from trajGen_grid_world import GridWorld_trajGen
from utils import utils

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [249]:
reward_mat = np.array([[ 1, 1, 1, 1, 1, 1, 1, 1, 2, 2],
                       [-1,-1, 1, 1, 1, 1, 1, 1, 1, 2],
                       [-2,-1,-1,-1,-1,-1,-1, 1, 1, 1],
                       [-2,-2,-2,-2,-2,-2,-2,-1,-1,-1],
                       [-1,-1,-1,-1,-1, 1,-2,-2,-2,-2],
                       [ 0, 0, 0, 1, 1, 2, 1,-1,-1,-1],
                       [ 0, 0, 1, 1, 2, 2, 2, 1,-1,-1],
                       [ 1, 1, 1, 1, 1, 1, 1, 0,-1,-1],
                       [ 2, 1, 1, 1, 1, 1, 0, 0,-1,-1],
                       [ 2, 2, 1, 1, 1, 1, 0,-1,-1,-2],])
reward_mat = utils.Normalize_2DArr(reward_mat)
world_trajGen = GridWorld_trajGen(10,10,reward_mat,trans_prob=0.8,discount=0.8)
world_trajGen.ShowRewards()

In [250]:
world_trajGen.feature_matrix(discrete=False).__len__()

100

In [251]:
world_trajGen.GenerateTrajectories(traj_count=500,traj_length=30,save=True)

100%|██████████| 500/500 [00:00<00:00, 3276.46it/s]


,m,trajs
0,1,"[(59, 3, 60), (60, 1, 70), (70, 1, 80), (80, 1..."
1,2,"[(1, 3, 2), (2, 3, 3), (3, 3, 4), (4, 3, 14), ..."
2,3,"[(12, 2, 2), (2, 3, 3), (3, 3, 4), (4, 3, 5), ..."
3,4,"[(51, 1, 52), (52, 1, 51), (51, 1, 61), (61, 1..."
4,5,"[(6, 3, 7), (7, 3, 8), (8, 0, 8), (8, 0, 8), (..."
...,...,...
495,496,"[(86, 2, 87), (87, 4, 86), (86, 2, 87), (87, 4..."
496,497,"[(8, 0, 8), (8, 0, 8), (8, 0, 9), (9, 0, 9), (..."
497,498,"[(29, 2, 19), (19, 2, 9), (9, 0, 9), (9, 0, 9)..."
498,499,"[(62, 3, 63), (63, 3, 64), (64, 3, 65), (65, 0..."


In [252]:
traj_path = r'demo_expert_trajs_0109.csv'
world = GridWorld(
    states_features=world_trajGen.feature_matrix(discrete=False),
    expert_traj_filePath=traj_path,
    width=10,
    height=10,
    active_all=True
)

In [253]:
world.ShowGridWorld_Count_log()

In [254]:
dme = DMEIRL(world,layers = (16,16),lr = 0.001)

In [255]:
rewards = dme.train(n_epochs=500,demo=True)

100%|██████████| 500/500 [35:10<00:00,  4.22s/it]


In [256]:
rewards.shape

torch.Size([100])

In [257]:
rewards = rewards.cpu().numpy().reshape(10,10)

In [259]:
world.ShowRewardsResult(rewards)